# HW 2: Language Modeling

In this homework you will be building several varieties of language models.

## Goal

We ask that you construct the following models in PyTorch:

1. A trigram model with linear-interpolation. $$p(y_t | y_{1:t-1}) =  \alpha_1 p(y_t | y_{t-2}, y_{t-1}) + \alpha_2 p(y_t | y_{t-1}) + (1 - \alpha_1 - \alpha_2) p(y_t) $$
2. A neural network language model (consult *A Neural Probabilistic Language Model* http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)
3. An LSTM language model (consult *Recurrent Neural Network Regularization*, https://arxiv.org/pdf/1409.2329.pdf) 
4. Your own extensions to these models...


Consult the papers provided for hyperparameters.

 


## Setup

This notebook provides a working definition of the setup of the problem itself. You may construct your models inline or use an external setup (preferred) to build your system.

In [ ]:
# Text text processing library
import torchtext
from torchtext.vocab import Vectors

The dataset we will use of this problem is known as the Penn Treebank (http://aclweb.org/anthology/J93-2004). It is the most famous dataset in NLP and includes a large set of different types of annotations. We will be using it here in a simple case as just a language modeling dataset.

To start, `torchtext` requires that we define a mapping from the raw text data to featurized indices. These fields make it easy to map back and forth between readable data and math, which helps for debugging.

In [ ]:
# Our input $x$
TEXT = torchtext.data.Field()

Next we input our data. Here we will use the first 10k sentences of the standard PTB language modeling split, and tell it the fields.

In [ ]:
# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

The data format for language modeling is strange. We pretend the entire corpus is one long sentence.

In [ ]:
print('len(train)', len(train))

Here's the vocab itself. (This dataset has unk symbols already, but torchtext adds its own.)

In [ ]:
TEXT.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))

When debugging you may want to use a smaller vocab size. This will run much faster.

In [ ]:
if False:
    TEXT.build_vocab(train, max_size=1000)
    len(TEXT.vocab)

The batching is done in a strange way for language modeling. Each element of the batch consists of `bptt_len` words in order. This makes it easy to run recurrent models like RNNs. 

In [ ]:
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=10, device=-1, bptt_len=32, repeat=False)

Here's what these batches look like. Each is a string of length 32. Sentences are ended with a special `<eos>` token.

In [ ]:
it = iter(train_iter)
batch = next(it) 
print(vars(batch))
print("Size of text batch [max bptt length, batch size]", batch.text.size())
print("Second in batch", batch.text[:, 2])
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 2].data]))

The next batch will be the continuation of the previous. This is helpful for running recurrent neural networks where you remember the current state when transitioning.

In [ ]:
batch = next(it)
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 2].data]))
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 3].data]))
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 4].data]))

There are no separate labels. But you can just use an offset `batch.text[1:]` to get the next word.

## Assignment

Now it is your turn to build the models described at the top of the assignment. 

Using the data given by this iterator, you should construct 3 different torch models that take in batch.text and produce a distribution over the next word. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/cs287-hw2-s18

For the final Kaggle test, we will have you do a next word prediction task. We will provide a 10 word prefix of sentences, and it is your job to predict 10 possible next word candidates

In [ ]:
!head input.txt

As a sample Kaggle submission, let us build a simple unigram model.  

In [ ]:
from collections import Counter
count = Counter()
for b in iter(train_iter):
    count.update(b.text.view(-1).data.tolist())
count[TEXT.vocab.stoi["<eos>"]] = 0
predictions = [TEXT.vocab.itos[i] for i, c in count.most_common(20)]
print(predictions)
with open("sample.txt", "w") as fout: 
    print("id,word", file=fout)
    for i, l in enumerate(open("input.txt"), 1):
        print("%d,%s"%(i, " ".join(predictions)), file=fout)


In [ ]:
!head sample.txt

The metric we are using is mean average precision of your 20-best list. 

$$MAP@20 = \frac{1}{|D|} \sum_{u=1}^{|D|} \sum_{k=1}^{20} Precision(u, 1:k)$$

Ideally we would use log-likelihood or ppl as discussed in class, but this is the best Kaggle gives us. This takes into account whether you got the right answer and how highly you ranked it. 

In particular, we ask that you do not game this metric. Please submit *exactly 20* unique predictions for each example.


As always you should put up a 5-6 page write-up following the template provided in the repository:  https://github.com/harvard-ml-courses/cs287-s18/blob/master/template/

# Neural Probabalistic Language Model

In [1]:
# Text text processing library
import torchtext
import torch
import math 
import torch.nn as nn 
from torchtext.vocab import Vectors
DEBUG = True 
# Our input $x$
TEXT = torchtext.data.Field()
# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)
TEXT.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))

if DEBUG == True:
    TEXT.build_vocab(train, max_size=1000)
    len(TEXT.vocab)
    print('len(TEXT.vocab)', len(TEXT.vocab))
    
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=12, device=-1, bptt_len=32, repeat=False, shuffle=False)

len(TEXT.vocab) 10001
len(TEXT.vocab) 1002


In [13]:
class NPLM(nn.Module):
    def __init__(self, V_vocab_dim, M_embed_dim, H_hidden_dim, N_seq_len):
        super(NPLM, self).__init__()
        self.vocab_size = V_vocab_dim
        self.embed = nn.Embedding(V_vocab_dim, M_embed_dim)
        self.hidden_linear = nn.Linear(M_embed_dim*N_seq_len, H_hidden_dim, bias=True)
        self.tanh_act = nn.Tanh()
        self.U_linear = nn.Linear(H_hidden_dim, V_vocab_dim, bias=True)
        self.W_linear = nn.Linear(M_embed_dim*N_seq_len, V_vocab_dim, bias=True)
        self.softmax = nn.LogSoftmax()

    def forward(self, x):
        x_embed = self.embed(x)
        x_flat = torch.autograd.Variable(torch.cat(x_embed.data, dim=1))
        hidden_feat = self.hidden_linear(x_flat)
        hidden_act = self.tanh_act(hidden_feat)
        hidden_out = self.U_linear(hidden_act)
        direct = self.W_linear(x_flat)
        out = direct + hidden_out
        return self.softmax(out)

In [6]:
def evaluate(model, data_iterator):
    # Turn on evaluation mode which disables dropout.
    #model.eval()
    total_loss = 0
    batch_count = 0
    for batch in iter(data_iterator):
        batch_loss = 0
        N_tokens = batch.text.size(0) - 4
        for i in range(N_tokens):
            output = model(batch.text[i:i+4])
            target = batch.text[i+4]
            batch_loss += criterion(output, target).data[0]
        total_loss += batch_loss/N_tokens
        batch_count += 1
    return total_loss / batch_count

def train_batch(model, criterion, optim, batch, label):
    # initialize hidden vectors
    model.zero_grad()
    # calculate forward pass
    y = model(batch)
    # calculate loss
    loss = criterion(y, label)
    # backpropagate and step
    loss.backward()
    optim.step()
    return loss.data[0]

# training loop
def run_training(model, criterion, optim, data_iterator):

    for e in range(n_epochs):
        batches = 0
        epoch_loss = 0
        for batch in iter(data_iterator):
            batch_loss = 0
            N_tokens = batch.text.size(0) - 4
            for i in range(N_tokens):
                batch_loss += train_batch(model, criterion, optim, batch.text[i:i+4], batch.text[i+4])
            batches += 1
            epoch_loss += batch_loss/N_tokens
        epoch_loss /= batches
        print("Epoch ", e, " Loss: ", epoch_loss, "Perplexity: ", math.exp(epoch_loss))
        train_loss = evaluate(model, data_iterator)
        print("Epoch Train Loss: ", train_loss, "Perplexity: ", math.exp(train_loss))
        val_loss = evaluate(model, val_iter)
        print("Epoch Val Loss: ", val_loss, "Perplexity: ", math.exp(val_loss))
        torch.save(npl.state_dict(), 'npl_full_model.pt')

In [8]:
# size of the embeddings and vectors
n_embedding = 30
n_hidden = 100
seq_len = 4
n_epochs = 10
learning_rate = .1
weight_decay = 0

npl = NPLM(len(TEXT.vocab), n_embedding, n_hidden, seq_len)
criterion = nn.NLLLoss()
optim = torch.optim.SGD(npl.parameters(), lr=learning_rate, weight_decay=weight_decay)

run_training(npl, criterion, optim, train_iter)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: generator 'BPTTIterator.__iter__' raised StopIteration


Epoch  0  Loss:  4.2661785848759735 Perplexity:  71.24884331745294


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'BPTTIterator.__iter__' raised StopIteration
  


Epoch Train Loss:  4.1354655157986 Perplexity:  62.51868773430979
Epoch Val Loss:  4.244445357805354 Perplexity:  69.71708138343561
Epoch  1  Loss:  4.105593250865666 Perplexity:  60.67873158191939
Epoch Train Loss:  4.0666462902115 Perplexity:  58.360908471332415
Epoch Val Loss:  4.207913405316392 Perplexity:  67.21614054136651
Epoch  2  Loss:  4.059195415375193 Perplexity:  57.92768459962089
Epoch Train Loss:  4.039158546874889 Perplexity:  56.778546213842596
Epoch Val Loss:  4.200333999228601 Perplexity:  66.70860794407817
Epoch  3  Loss:  4.0322333836226765 Perplexity:  56.38670386420162
Epoch Train Loss:  4.023240508314374 Perplexity:  55.88189849521274
Epoch Val Loss:  4.200041637254768 Perplexity:  66.68910773448677
Epoch  4  Loss:  4.014770594938606 Perplexity:  55.41058247626998
Epoch Train Loss:  4.007050063105906 Perplexity:  54.984430487179715
Epoch Val Loss:  4.196109267317041 Perplexity:  66.42737644146621
Epoch  5  Loss:  4.001994693432386 Perplexity:  54.70716529430267


KeyboardInterrupt: 

In [14]:
with open("sample.txt", "w") as fout: 
    print("id,word", file=fout)
    for i, l in enumerate(open("input.txt"), 1):
        print(l)
        input_tokens = l.split(" ")[-5:-1]
        input_index = torch.LongTensor([TEXT.vocab.stoi[t] for t in input_tokens]).unsqueeze(1)
        output = npl(torch.autograd.Variable(input_index))
        max_values, max_indices = torch.topk(output, 20)
        print(" ".join([TEXT.vocab.itos[int(i)] for i in max_indices.data[0, :]]))
        
        #print("%d,%s"%(i, " ".join(predictions)), file=fout)

but while the new york stock exchange did n't fall ___

<eos> on <unk> and closed face for out said a that had as about of will much set to were
some circuit breakers installed after the october N crash failed ___

to rates rate N <unk> by the shares 's at or and business with for as this stock which european
the N stock specialist firms on the big board floor ___

the <unk> in as a <eos> its to N and $ are his 's an that of about do at
big investment banks refused to step up to the plate ___

<eos> <unk> of for law and in N to has the 's are or stock was year public with trade
heavy selling of standard & poor 's 500-stock index futures ___

contract a markets <unk> the for <eos> programs prices terms off problems in $ and no were news some has
seven big board stocks ual amr bankamerica walt disney capital ___

<eos> markets <unk> at for management by gains inc. in and a as corp. to which the losses is co.
once again the specialists were not able to handle the ___

<unk> first new fede

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



in any case on the day of the meeting quantum ___

<unk> <eos> the and 's said which for a of how held that or group to common in is has
on top of everything else quantum <unk> a disaster at ___

least a the <unk> $ to this N first about an its best last by all home it money more
after an explosion <unk> the plant in june the company ___

's was will said has is <eos> also reported to could had would <unk> of agreed can that for expects
this human toll adds the most painful <unk> yet to ___

a $ the <unk> meet N acquire mr. make an do produce be his help hold buy reduce not yield
until this year the company had been steadily lowering its ___

<unk> offer stake own the N business investment european huge as three board shares operating <eos> previous earlier parent third-quarter
a prolonged production halt at the plant could introduce another ___

<unk> to at a and or dollars <eos> by shares that this such the said dec. of in medical you
when a plant has just been running flat out to _


<unk> net $ top a office management board N move u.s. the action recent price budget agreement earnings investors very
bozell joins backer spielvogel bates and ogilvy group as u.s. ___

<unk> as <eos> not and at money for sales of a in companies result manager attorney to retail by big
citing a payment from a supplier and strong sales of ___

<unk> its units the about new economic debt N $ this a real assets japan american shares volume senior u.s.
the maker of <unk> products said net income rose to ___

$ <unk> N its the a acquire report build provide sell do close pay have meet increase his be purchase
<unk> said its results were boosted by $ N million ___

<eos> of and or in from for to a <unk> that on which at loss said 's up compared down
<unk> said effects from <unk> the line may have a ___

<unk> far major N gain special computer takeover new spokesman certain few little leading stake way car problem strong close
a spokeswoman would n't elaborate but the company said the ___

<

of $ market for but <unk> <eos> with a that to said and financial statement securities it by issue area
to <unk> itself <unk> is also expanding international coverage and ___

<unk> a other the N not by is to $ related losses construction at provide for no food loan so
it is paying higher salaries after years of <unk> to ___

<unk> the do have a provide be use see reduce mr. N report this meet sell los it his allow
and it is <unk> on an expensive gamble to break ___

the a <eos> <unk> by its an for with and their losses as at $ about in N so back
the next stage is to get beyond the opinion leaders ___

to <eos> the is in a as <unk> for this of by and on at has 's maker bond mr.
networks like other consumer products develop images in peoples ' ___

future <unk> position <eos> $ financial for problems bid business meeting offer of a but and said by role investment
it also takes money that <unk> has been reluctant to ___

<unk> N $ be the a provide see his acquire deal its purchase positi

...


a <unk> any <eos> N this had the an some sold is it given own said how made of board
margin calls since friday have been higher than usual but ___

the it to <unk> that <eos> he they in for are with as not one and only i other later
merrill lynch & co. officials do n't expect margin calls ___

for to <unk> in of <eos> yesterday on the this co. by and said is a after 's that unit
hugo <unk> senior vice president at charles schwab corp. the ___

<unk> recent the new school u.s. market maker loan parent a offering government last insurance company federal subject chicago european
he said schwab had increased margin requirements so customers have ___

<unk> a n't long an the been would already mr. made for no his enough set offered few raised to
<unk> inc. following rival hertz corp. 's lead said it ___

will did is has <unk> wo would expects also received had to <eos> was plans acquired still said 's filed
the garden city n.y. <unk> company said it wo n't ___

<unk> be make comment the h

a daily <unk> program called the march of time tries ___

<unk> <eos> in by of a and the 's to ' for from at N national or $ said treasury
and to attract younger <unk> radio free europe <unk> the ___

<unk> new most first offer N federal proposed shares test government trust way treasury stock same industry u.s. general company
we are <unk> for all the news says mr. <unk> ___

<eos> said as and <unk> with is of a for $ at its 's has that notes to recently had
proposals for <unk> national service like <unk> <unk> up from ___

a the $ <unk> N an its his at their # one creditors such it <eos> about what for out
the disease <unk> comes to mind of course not as ___

<unk> the a low of an to it much they well part soon on expected is $ for being merrill
rather it is born of frustration with having to combat ___

the <unk> its their <eos> with about by and a to N that in it as for them an higher
it is back with us again in the form of ___

<unk> economic a its this debt assets N $ the chicago

the <eos> <unk> 's in with management and by which of for that or are national has corp. an policy
to be successful a product can be any color <unk> ___

<unk> led inc. said of are <eos> the which and to in posted by would a co. it has N
by last winter justin was showing <unk> at toy <unk> ___

<eos> by and the <unk> of a 's inc. for in to university co. this which an as said has
indeed concerned that sony sales personnel were threatening legal action ___

<eos> of for to a in with when the it that by <unk> from at 's profit an came on
he himself threatened to take the matter to the federal ___

reserve government <unk> budget agency <eos> law trade or 's court average and securities judge national company was a programs
but justin has n't pursued those charges which were without ___

the <unk> a $ it one N programs products stock of stocks all in about having interest <eos> any orders
recalls mr. <unk> our purpose was to influence them to ___

<unk> a provide the do N acquire $ yield 

the to is <eos> for on of and a it this about center with has will as its by since
the court is saying that the adverse facts have to ___

a <unk> the $ make do this N sell yield no turn control be reduce his those lower its pay
shareholders ' attorneys at the new york firm of <unk> ___

<unk> the in stock with for 's that <eos> to said inc. management charge as of by and net corp.
they wrote the opinion <unk> a new rule of <unk> ___

<eos> <unk> the management 's in which by and an for or with stock of at into loans said work
nfl ordered to pay $ N million in legal fees ___

's via to said is N business and reserves <eos> <unk> which weeks income costs securities on that was for
the national football league is considering appealing the ruling stemming ___

<eos> by and of for a <unk> to said the is 's which at that or with rate has system
a jury in N agreed with the <unk> 's claims ___

<eos> <unk> with at the index said are had and is for were 's law earthquake a an outstanding accou

the chinese leaders have to decide whether they want control ___

of the <unk> shares corp. data over for more <eos> they interest or ago group N investor said corp and
the bush administration urging the supreme court to give states ___

<unk> N of from new in as also air but and after while stock year level ' <eos> for two
<unk> general kenneth <unk> argued that the N supreme court ___

to <eos> <unk> for in at on of or is and N with the 's stocks said as into by
he also argued that the high court was wrong in ___

<unk> the a its N new august other his federal several april europe this their growth court such part early
the administration 's position was outlined in a <unk> brief ___

<unk> to <eos> of the and for said in by 's test & has between would on is notes tax
the administration filed the brief in an appeal involving a ___

<unk> unit new N government university share huge percentage takeover significant capital london little large recent maker back $ vice
the administration 